# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [1]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [2]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [17]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);

In [18]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.22876,0.0,8.56,0,0.52,6.405,85.4,2.7147,5,384
2,6.39312,0.0,18.1,0,0.584,6.162,97.4,2.206,24,666
3,7.05042,0.0,18.1,0,0.614,6.103,85.1,2.0218,24,666
4,0.03768,80.0,1.52,0,0.404,7.274,38.3,7.309,2,329
5,9.82349,0.0,18.1,0,0.671,6.794,98.8,1.358,24,666
6,8.05579,0.0,18.1,0,0.584,5.427,95.4,2.4298,24,666
7,0.01951,17.5,1.38,0,0.4161,7.104,59.5,9.2229,3,216
8,2.14918,0.0,19.58,0,0.871,5.709,98.5,1.6232,5,403
9,1.41385,0.0,19.58,1,0.871,6.129,96.0,1.7494,5,403


In [13]:
names(boston)

14-element Array{Symbol,1}:
 :Crim   
 :Zn     
 :Indus  
 :Chas   
 :NOx    
 :Rm     
 :Age    
 :Dis    
 :Rad    
 :Tax    
 :PTRatio
 :Black  
 :LStat  
 :MedV   

In [19]:
ols = GLM.lm(@formula(Crim ~ 
        Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + 
        Tax + PTRatio + Black + LStat + MedV), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Crim ~ 1 + Zn + Indus + Chas + NOx + Rm + Age + Dis + Rad + Tax + PTRatio + Black + LStat + MedV

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────
                  Estimate  Std. Error     t value  Pr(>|t|)    Lower 95%    Upper 95%
──────────────────────────────────────────────────────────────────────────────────────
(Intercept)   17.7255       8.67495      2.0433       0.0417    0.670136   34.7809    
Zn             0.0494975    0.0232565    2.12833      0.0339    0.003774    0.095221  
Indus         -0.0976118    0.106035    -0.920561     0.3578   -0.306082    0.110859  
Chas          -0.572903     1.45433     -0.393929     0.6938   -3.43219     2.28638   
NOx          -10.932        6.44883     -1.69519      0.0908  -23.6107      1.74671   
Rm             0.85444

In [8]:
predict(ols, test)

101-element Array{Union{Missing, Float64},1}:
  0.4938516310325465
  1.632659800332978 
  1.9285542312313124
  9.191444639242146 
 13.816423159192974 
 -0.5237416483955455
 18.4394104255622   
  3.767050983359997 
  0.422461280217048 
  2.818488775886588 
 -0.7542114686300536
  2.6451829431935927
  0.8467476692539169
  ⋮                 
 -1.8008229304223766
 15.423759199541916 
  0.9842985833327376
 -0.7212260134843884
 -0.2775802657493802
 -0.7726612321006314
  4.372508982493573 
 -4.459436532842503 
  0.5936311466644622
  9.78477387593924  
 15.513355203545945 
 -4.061155227004143 

In [20]:
GLM.r²(ols)

0.44440183074401773

In [21]:
GLM.adjr²(ols)

0.425929257341645

In [25]:
descr(boston)

UndefVarError: UndefVarError: descr not defined